In [ ]:
!pip install "transformers[torch]" pandas datasets numpy wandb

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("fill-mask", model="FacebookAI/xlm-roberta-base")

pipe.predict("Estamos intentando predecir si el siguiente texto contiene sesgo hacia las mujeres, misoginia o machismo. Consideramos que el texto sí es machista si el texto incluye la sobrevalorización de la fuerza física sobre la inteligencia, la oposición a las mujeres, la promoción de la desigualdad, la explotación y la falta de respeto. El texto es: '@_betterton @juanrallo para cometer atentados terroristas (2) no creo que entonces existiera el burkini'. El texto <mask> es machista.")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

KeyboardInterrupt: 

## Few-shot fine tuning

In [1]:
from datasets import Dataset, DatasetDict, load_metric
import numpy as np
import pandas as pd
from transformers import (
    AutoTokenizer,
    AutoModelForMaskedLM,
    DefaultDataCollator,
    TrainingArguments,
    EarlyStoppingCallback,
    Trainer,
)
import wandb

In [2]:
# Get model and tokenizer as global variables
model_name = "dccuchile/bert-base-spanish-wwm-cased"
run_name = "few_shot_training"
project_name = "BETO-Prompting-Few-Shot-Gender-Bias-Detection"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

model = AutoModelForMaskedLM.from_pretrained(
    model_name,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [3]:
wandb.init(project=project_name, name=run_name)

wandb: Currently logged in as: ignacioct. Use `wandb login --relogin` to force relogin


In [4]:
def tokenize_function(examples):
    return tokenizer(
        examples["text"], padding="max_length", max_length=512, truncation=True
    )

In [5]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


# Loading data
train_data = pd.read_csv("train_data.csv")
test_data = pd.read_csv("test_data.csv")
valid_data = pd.read_csv("val_data.csv")

train_data = train_data.rename(columns={"Contents": "text"})
test_data = test_data.rename(columns={"Contents": "text"})
valid_data = valid_data.rename(columns={"Contents": "text"})

train_data = train_data.rename(columns={"Label": "label"})
test_data = test_data.rename(columns={"Label": "label"})
valid_data = valid_data.rename(columns={"Label": "label"})

# Drop all columns that are not "text" and "label"
train_data = train_data[["text", "label"]]
test_data = test_data[["text", "label"]]
valid_data = valid_data[["text", "label"]]

# Sampling 10 positive examples and 10 negatives examples from the training dataset
train_data_sample = pd.DataFrame()
test_data_sample = pd.DataFrame()
valid_data_sample = pd.DataFrame()
train_data = train_data.sample(frac=1)  #shuffle

counter_positives = 20
counter_negatives = 20

for index, row in train_data.iterrows():
  if row["label"] == 1 and counter_positives>0:
    text_to_append = f"Predice si el siguiente texto contiene sesgo hacia las mujeres, misoginia o machismo. Consideramos que el texto sí es machista si el texto incluye la sobrevalorización de la fuerza física sobre la inteligencia, la oposición a las mujeres, la promoción de la desigualdad, la explotación y la falta de respeto. El texto es: '{row['text']}'. El texto sí es machista."
    train_data_sample = train_data_sample.append({"text": text_to_append}, ignore_index=True)
    counter_positives -=1
  elif row["label"] == 0 and counter_negatives>0:
    text_to_append = f"Predice si el siguiente texto contiene sesgo hacia las mujeres, misoginia o machismo. Consideramos que el texto sí es machista si el texto incluye la sobrevalorización de la fuerza física sobre la inteligencia, la oposición a las mujeres, la promoción de la desigualdad, la explotación y la falta de respeto. El texto es: '{row['text']}'. El texto no es machista."
    train_data_sample = train_data_sample.append({"text": text_to_append}, ignore_index=True)
    counter_negatives -=1

for index, row in test_data.iterrows():
  if row["label"] == 1 :
    text_to_append = f"Predice si el siguiente texto contiene sesgo hacia las mujeres, misoginia o machismo. Consideramos que el texto sí es machista si el texto incluye la sobrevalorización de la fuerza física sobre la inteligencia, la oposición a las mujeres, la promoción de la desigualdad, la explotación y la falta de respeto. El texto es: '{row['text']}'. El texto sí es machista."
    test_data_sample = test_data_sample.append({"text": text_to_append}, ignore_index=True)
  elif row["label"] == 0 :
    text_to_append = f"Predice si el siguiente texto contiene sesgo hacia las mujeres, misoginia o machismo. Consideramos que el texto sí es machista si el texto incluye la sobrevalorización de la fuerza física sobre la inteligencia, la oposición a las mujeres, la promoción de la desigualdad, la explotación y la falta de respeto. El texto es: '{row['text']}'. El texto no es machista."
    test_data_sample = test_data_sample.append({"text": text_to_append}, ignore_index=True)

for index, row in valid_data.iterrows():
  if row["label"] == 1 :
    text_to_append = f"Predice si el siguiente texto contiene sesgo hacia las mujeres, misoginia o machismo. Consideramos que el texto sí es machista si el texto incluye la sobrevalorización de la fuerza física sobre la inteligencia, la oposición a las mujeres, la promoción de la desigualdad, la explotación y la falta de respeto. El texto es: '{row['text']}'. El texto sí es machista."
    valid_data_sample = valid_data_sample.append({"text": text_to_append}, ignore_index=True)
  elif row["label"] == 0 :
    text_to_append = f"Predice si el siguiente texto contiene sesgo hacia las mujeres, misoginia o machismo. Consideramos que el texto sí es machista si el texto incluye la sobrevalorización de la fuerza física sobre la inteligencia, la oposición a las mujeres, la promoción de la desigualdad, la explotación y la falta de respeto. El texto es: '{row['text']}'. El texto no es machista."
    valid_data_sample = valid_data_sample.append({"text": text_to_append}, ignore_index=True)

print(len(train_data_sample))
print(len(valid_data_sample))
print(len(test_data_sample))

train_data_sample.head(20)

40
288
287


,text
0,Predice si el siguiente texto contiene sesgo h...
1,Predice si el siguiente texto contiene sesgo h...
2,Predice si el siguiente texto contiene sesgo h...
3,Predice si el siguiente texto contiene sesgo h...
4,Predice si el siguiente texto contiene sesgo h...
5,Predice si el siguiente texto contiene sesgo h...
6,Predice si el siguiente texto contiene sesgo h...
7,Predice si el siguiente texto contiene sesgo h...
8,Predice si el siguiente texto contiene sesgo h...
9,Predice si el siguiente texto contiene sesgo h...


In [6]:
# Create a dataset dictionary
dataset_dict = {
    "train": Dataset.from_pandas(train_data_sample),
    "valid": Dataset.from_pandas(valid_data_sample),
    "test": Dataset.from_pandas(valid_data_sample)
}

# Turn it into HuggingFace dataset
dataset = DatasetDict(dataset_dict)

dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 40
    })
    valid: Dataset({
        features: ['text'],
        num_rows: 288
    })
    test: Dataset({
        features: ['text'],
        num_rows: 288
    })
})

In [7]:
tokenizer.decode(1112)

's i'

In [8]:
tokenizer.decode(1084)

'n o'

In [9]:
tokenizer.decode(0)

'[ M A S K ]'

In [10]:
# Tokenize the data
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/288 [00:00<?, ? examples/s]

Map:   0%|          | 0/288 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 40
    })
    valid: Dataset({
        features: ['text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 288
    })
    test: Dataset({
        features: ['text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 288
    })
})

In [11]:
def add_labels_fix_mask_tokens(example):
    example["labels"] = example["input_ids"]
    example["input_ids"] = [0 if x==1112 or x==1084 else x for x in example["input_ids"]]
    return example

In [12]:
tokenized_dataset = tokenized_dataset.map(add_labels_fix_mask_tokens)

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/288 [00:00<?, ? examples/s]

Map:   0%|          | 0/288 [00:00<?, ? examples/s]

In [13]:
tokenized_dataset["train"][0]

{'text': "Predice si el siguiente texto contiene sesgo hacia las mujeres, misoginia o machismo. Consideramos que el texto sí es machista si el texto incluye la sobrevalorización de la fuerza física sobre la inteligencia, la oposición a las mujeres, la promoción de la desigualdad, la explotación y la falta de respeto. El texto es: '@koenmajr20 probando nuevos morning show! @exafm me aburre ya! se ha vuelto demasiado tonto el programa! extraño a @cecidelacueva'. El texto no es machista.",
 'input_ids': [4,
  2583,
  6062,
  0,
  1040,
  2435,
  3881,
  6190,
  2348,
  1123,
  2333,
  1089,
  2409,
  1017,
  1354,
  30933,
  4874,
  1255,
  1074,
  23885,
  1898,
  1009,
  30250,
  1038,
  1040,
  3881,
  1794,
  1058,
  23885,
  1629,
  0,
  1040,
  3881,
  5091,
  1030,
  1269,
  18985,
  2502,
  1008,
  1030,
  3920,
  6929,
  1269,
  1030,
  9145,
  1017,
  1030,
  8277,
  1013,
  1089,
  2409,
  1017,
  1030,
  5195,
  1008,
  1030,
  20536,
  1017,
  1030,
  6086,
  1042,
  1030,
  

In [14]:
# Defining the Data Collator
data_collator = DefaultDataCollator()

 # Training
training_args = TrainingArguments(
    output_dir="output",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=1,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    report_to="wandb",
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["valid"],
    tokenizer=tokenizer,
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,15.532900,15.879785
2,4.525600,2.048736
3,0.469500,0.354926
4,0.165400,0.139718
5,0.109500,0.093147


Checkpoint destination directory output/checkpoint-5 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory output/checkpoint-10 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory output/checkpoint-15 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory output/checkpoint-20 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory output/checkpoint-25 already exists and is non-empty. Saving will proceed but saved results may be invalid.


TrainOutput(global_step=25, training_loss=5.316130428314209, metrics={'train_runtime': 64.6207, 'train_samples_per_second': 3.095, 'train_steps_per_second': 0.387, 'total_flos': 52641257472000.0, 'train_loss': 5.316130428314209, 'epoch': 5.0})

In [ ]:
trainer.evaluate(tokenized_dataset["test"])

{'eval_loss': 0.01631937548518181,
 'eval_runtime': 4.8682,
 'eval_samples_per_second': 59.16,
 'eval_steps_per_second': 7.395,
 'epoch': 10.0}

In [17]:
trainer.save_model("beto-fill-mask-fine-tuned")

In [20]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("fill-mask", model="beto-fill-mask-fine-tuned", targets=["no", "sí"])

pipe.predict("Predice si el siguiente texto contiene sesgo hacia las mujeres, misoginia o machismo. Consideramos que el texto sí es machista si el texto incluye la sobrevalorización de la fuerza física sobre la inteligencia, la oposición a las mujeres, la promoción de la desigualdad, la explotación y la falta de respeto. El texto es: 'acabo de etiquetar a mi tía en una cosa. soy más allá de mi propia tía. soy la tía de las tías.'. El texto [MASK] es machista.")

[{'score': 0.4389346241950989,
  'token': 1084,
  'token_str': 'no',
  'sequence': 'Predice si el siguiente texto contiene sesgo hacia las mujeres, misoginia o machismo. Consideramos que el texto sí es machista si el texto incluye la sobrevalorización de la fuerza física sobre la inteligencia, la oposición a las mujeres, la promoción de la desigualdad, la explotación y la falta de respeto. El texto es : acabo de etiquetar a mi tía en una cosa. soy más allá de mi propia tía. soy la tía de las tías.. El texto no es machista.'},
 {'score': 0.19711056351661682,
  'token': 1794,
  'token_str': 'sí',
  'sequence': 'Predice si el siguiente texto contiene sesgo hacia las mujeres, misoginia o machismo. Consideramos que el texto sí es machista si el texto incluye la sobrevalorización de la fuerza física sobre la inteligencia, la oposición a las mujeres, la promoción de la desigualdad, la explotación y la falta de respeto. El texto es : acabo de etiquetar a mi tía en una cosa. soy más allá de mi 

In [28]:
# Loading data
test_data = pd.read_csv("test_data.csv")
test_data = test_data.rename(columns={"Contents": "text"})
test_data = test_data.rename(columns={"Label": "label"})
test_data = test_data[["text", "label"]]

In [30]:
gold_labels=[]
predictions=[]
pipe = pipeline("fill-mask", model="beto-fill-mask-fine-tuned", targets=["no", "sí"])
for _, row in test_data.iterrows():
  gold_labels.append(row["label"])

  pred = pipe.predict(f"Predice si el siguiente texto contiene sesgo hacia las mujeres, misoginia o machismo. Consideramos que el texto sí es machista si el texto incluye la sobrevalorización de la fuerza física sobre la inteligencia, la oposición a las mujeres, la promoción de la desigualdad, la explotación y la falta de respeto. El texto es: '{row['text']}'. El texto [MASK] es machista.")

  predictions.append(pred[0]["token_str"])

print(predictions)

['no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no

In [31]:
predictions_binary = []

for pred in predictions:
  if pred == "no":
    predictions_binary.append(0)
  else:
    predictions_binary.append(1)

In [33]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

def calculate_metrics(gold_labels, predictions):
    # Calculate precision
    precision = precision_score(gold_labels, predictions, average='macro')

    # Calculate recall
    recall = recall_score(gold_labels, predictions, average='macro')

    # Calculate F1-score
    f1 = f1_score(gold_labels, predictions, average='macro')

    # Calculate accuracy
    accuracy = accuracy_score(gold_labels, predictions)

    return precision, recall, f1, accuracy

# Example usage:
precision, recall, f1, accuracy = calculate_metrics(gold_labels, predictions_binary)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("Accuracy:", accuracy)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision: 0.41289198606271776
Recall: 0.5
F1-score: 0.4522900763358778
Accuracy: 0.8257839721254355


In [34]:
import math

eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

>>> Perplexity: 1.10
